<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkST0151ENSkillsNetwork20531532-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>



#### Import the required libraries we need for the lab.


In [ ]:
import piplite
await piplite.install(['numpy'],['pandas'])
await piplite.install(['seaborn'])

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pyplot
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

#### Read the dataset in the csv file from the URL


In [ ]:
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv'
resp = await fetch(URL)
boston_url = io.BytesIO((await resp.arrayBuffer()).to_py())

In [ ]:
boston_df=pd.read_csv(boston_url)

#### Add your code below following the instructions given in the course to complete the peer graded assignment


## Task 1: Become familiar with the dataset
The following describes the dataset variables:

·      CRIM - per capita crime rate by town

·      ZN - proportion of residential land zoned for lots over 25,000 sq.ft.

·      INDUS - proportion of non-retail business acres per town.

·      CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)

·      NOX - nitric oxides concentration (parts per 10 million)

·      RM - average number of rooms per dwelling

·      AGE - proportion of owner-occupied units built prior to 1940

·      DIS - weighted distances to five Boston employment centres

·      RAD - index of accessibility to radial highways

·      TAX - full-value property-tax rate per $10,000

·      PTRATIO - pupil-teacher ratio by town

·      LSTAT - % lower status of the population

·      MEDV - Median value of owner-occupied homes in $1000's

In [ ]:
## The table below displays the first few rows of the data. This provides the structure of the data.
boston_df.head()

In [ ]:
## The table below displays the statisital overview of the data.
boston_df.describe()

## Task 2: Generate Descriptive Statistics and Visualizations

In [ ]:
pyplot.figure(figsize=(8,5))

sns.boxplot(x=boston_df.MEDV)
pyplot.title("Median Value of Owner-Occupied Homes")
pyplot.xlabel("Value in $1000's")


ax = sns.boxplot(x="MEDV", data=boston_df)
pyplot.show()

The distrubution of the median values of owner-occupied homes is not symmetric, suggesting that the majority of houses are in the low to middle range.

In [ ]:
CHAS_count = boston_df.groupby('CHAS')[['Unnamed: 0']].count().reset_index()

sns.set(style="whitegrid")
ax = sns.barplot(x="CHAS", y="Unnamed: 0", data=CHAS_count)
pyplot.title("Numbers of Houses Bounded by the Charles River or Not")
pyplot.xlabel("Variables (1 if tract bounds river; 0 otherwise)")
pyplot.ylabel("Counts")

pyplot.show()

It is clear from the figure that the numbers of houses bounded by Charles river is small compared to those not bounded.

In [ ]:
boston_df.loc[(boston_df['AGE'] <= 35), 'age_group'] = '<=35 years'
boston_df.loc[(boston_df['AGE'] > 35) & (boston_df['AGE'] < 70), 'age_group'] = '35--70 years'
boston_df.loc[(boston_df['AGE'] >= 70), 'age_group'] = '>=70 years'

boston_df.age_group.reset_index()

ax = sns.boxplot(x="age_group", y="MEDV", data=boston_df)
pyplot.title("Median Value of Owner-Occupied Homes Grouped by Ages Grpups")
pyplot.xlabel("Age Groups")
pyplot.ylabel("Value in $1000's")

pyplot.show()

In [ ]:
ax = sns.scatterplot(y='NOX', x='INDUS', data=boston_df)

pyplot.title("Relationship between Nitric Oxide Concentrations and\n The Proportion of Non-Retail Business Acres Per Town")
pyplot.ylabel("Nitric Oxide Concentrations")
pyplot.xlabel("The Proportion of\n Non-Retail Business Acres Per Town")

pyplot.show()

The ilustration suggests the positive correlation between between Nitric oxide concentrations and the proportion of non-retail business acres per town, i.e. as the proportion of non-retail business acres per town increase, Nitric oxide concentrations also increase.

In [ ]:
sns.displot(boston_df['PTRATIO'], bins=10, kde = False)

pyplot.title("Numbers of Houses by Pupil to Teacher Ratio")
pyplot.ylabel("Counts")
pyplot.xlabel("Pupil to Teacher Ratio")

pyplot.show()

The number houses with pupil to teacher ratio at about 21:1 is the highest (mode).

## Task 3: Use the appropriate tests to answer the questions provided.

#### Is there a significant difference in median value of houses bounded by the Charles river or not? (T-test for independent samples)

State the hypothesis

*   $H\_0: µ\_1 = µ\_2$ ("there is no difference in median value of houses bounded by the Charles river or not")
*   $H\_1: µ\_1 ≠ µ\_2$ ("there is a difference in median value of houses bounded by the Charles river or not")

In [ ]:
scipy.stats.ttest_ind(boston_df[boston_df['CHAS'] == 0]['MEDV'],
                      boston_df[boston_df['CHAS'] == 1]['MEDV'], equal_var = True)

**Conclusion:** Since the p-value is less than alpha value 0.05, we reject the null hypothesis as there is enough proof that there is a statistical difference  in median value of houses bounded by the Charles river or not.

#### Is there a difference in Median values of houses (MEDV) for each proportion of owner occupied units built prior to 1940 (AGE)? (ANOVA)*

State the hypothesis

*   $H\_0: µ\_1 = µ\_2 = µ\_3$ (the three population means are equal)
*   $H\_1:$ At least one of the means differ

In [ ]:
medv_below_35 = boston_df[boston_df['age_group'] == '<=35 years']['MEDV']
medv_35 = boston_df[boston_df['age_group'] == '35--70 years']['MEDV']
medv_70_above = boston_df[boston_df['age_group'] == '>=70 years']['MEDV']

f_statistic, p_value = scipy.stats.f_oneway(medv_below_35, medv_35, medv_70_above)
print("F_Statistic: {0}, P-Value: {1}".format(f_statistic,p_value))

**Conclusion:** Since the p-value is less than alpha value 0.05, we reject the null hypothesis as there is enough proof that there is a statistical difference in Median values of houses (MEDV) for each proportion of owner occupied units built prior to 1940

#### Can we conclude that there is no relationship between Nitric oxide concentrations and proportion of non-retail business acres per town? (Pearson Correlation)

State the hypothesis:

*   $H\_0:$ Nitric oxide concentrations is not correlated with proportion of non-retail business acres per town
*   $H\_1:$ Nitric oxide concentrations is correlated with proportion of non-retail business acres per town

In [ ]:
scipy.stats.pearsonr(boston_df['NOX'], boston_df['INDUS'])

**Conclusion:** Since the p-value  < 0.05, we reject the Null hypothesis and conclude that there exists a relationship between Nitric oxide concentrations and proportion of non-retail business acres per town.

#### What is the impact of an additional weighted distance to the five Boston employment centres on the median value of owner occupied homes? (Regression analysis)

In [ ]:
X = boston_df['DIS']
y = boston_df['MEDV']
X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

model.summary()

**Conclusion:** p-value < 0.05 there is evidence of correlation between an additional weighted distance to the five Boston employment centres on the median value of owner occupied homes.